In [110]:
import ast
import json
import numpy as np
import pandas as pd

from jass.game.const import trump_strings_short

In [47]:
df = pd.read_json("data/jass_game_0001.txt", lines=True)

In [48]:
df.head()

,game,date,player_ids
0,"{'version': 'V0.2', 'trump': 1, 'dealer': 2, '...",2017-10-13 22:31:05,"[58663, 21516, 41630, 70654]"
1,"{'version': 'V0.2', 'trump': 1, 'dealer': 0, '...",2017-10-13 22:47:34,"[36972, 67618, 5202, 60659]"
2,"{'version': 'V0.2', 'trump': 3, 'dealer': 1, '...",2017-10-22 21:37:52,"[0, 0, 0, 0]"
3,"{'version': 'V0.2', 'trump': 2, 'dealer': 3, '...",2017-10-23 14:46:16,"[0, 0, 0, 0]"
4,"{'version': 'V0.2', 'trump': 1, 'dealer': 1, '...",2017-10-23 16:10:02,"[14476, 68780, 57175, 45129]"


In [56]:
def extract_game(df: pd.DataFrame) -> pd.DataFrame:
    return pd.json_normalize(df["game"])

In [102]:
def drop_unused_columns(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop([
        "player",
        "currentPlayer",
        "version",
        "jassTyp"
    ], axis=1)

In [103]:
def one_hot_trump(df: pd.DataFrame) -> pd.DataFrame:
    y = pd.get_dummies(df.trump, prefix="trump")
    df.drop(["trump"], axis=1, inplace=True)
    return pd.concat([df, y], axis=1)

In [125]:
def calc_player_hand(df: pd.DataFrame) -> pd.DataFrame:
    def __get_player_hands(row):
        tricks = row.tricks
        players = pd.Series([[] for _ in range(4)], index=[f"player_{i}_hand" for i in range(4)])
        for trick in tricks:
            cards, first = trick["cards"], trick["first"]
            for player_index, card in enumerate(cards):
                players[f"player_{(first + player_index) % 4}_hand"].append(card)
        return pd.concat([row, players])
                
    return df.apply(__get_player_hands, axis=1, result_type='expand')

In [267]:
def game_to_rounds(df: pd.DataFrame) -> pd.DataFrame:
    def __expand_df(raw_tricks):
        #print(raw_tricks)
        expanded_trick_rounds = pd.DataFrame(columns=[
            "player", 
            "winner", 
            "first", 
            "tricks"
        ])
        for game_round, trick in enumerate(raw_tricks):
            (cards, points, winner, first_player) = trick["cards"], trick["points"], trick["win"], trick["first"]
            for round_index, card in enumerate(cards):
                current_player = (first_player + round_index) % 4
                tricks = [r["cards"] for r in raw_tricks[:game_round]]
                tricks.append(["" for _ in range(4)])
                for i in range(round_index + 1):
                    #print(i)
                    #print(tricks[game_round])
                    tricks[game_round][i] = cards[i]

                expanded_trick_rounds.append({
                    "player": current_player,
                    "winner": winner,
                    "first_player": first_player,
                    "tricks": tricks
                }, ignore_index=True)
        return expanded_trick_rounds
        
    expanded_tricks = {k: __expand_df(v) for k, v in df.pop("tricks").items()}
    
    #expanded_trick_rounds = []
    
    #print(next(df.pop("tricks").items()))

            
    
    return df.join(expanded_tricks, rsuffix='_').reset_index(drop=True)

In [268]:
def one_hot_cards(df: pd.DataFrame) -> pd.DataFrame:
    df.tricks.apply()

In [269]:
(df[:100].pipe(extract_game)
     .pipe(drop_unused_columns)
     .pipe(one_hot_trump)
     .pipe(calc_player_hand)
     .pipe(game_to_rounds)
).head()

TypeError: Can only append a dict if ignore_index=True

In [ ]:
df['game'][0]